In [9]:
import pickle
import random

import sys
sys.path.append("./GIPS")
from core.MV2 import MV2
from core.JIG import JIG
from core.SG2 import SG2

In [10]:
def GIPS(str_feature,
		K, M, # MV2 파라미터
		 thetaJ,  # JIG 파라미터
		 vector_size, eps, minpts, hh1_size, hh2_size, ratio # SG2, AWL 파라미터
		):

	print(f'data no: {len(str_feature)}')

	str_feature = list(str_feature)
	for feature in str_feature:
		feature = list(feature)

	# 빅 그룹 식별
	minhashed_virtual_vectors = MV2(payloads=str_feature, K=K, M=M)

	big_group_indices = JIG(vectors=minhashed_virtual_vectors, thetaJ=thetaJ)
  
	
	big_group_payloads = []
	non_big_group_paylaods = []

	for idx, payload in enumerate(str_feature):
		if idx in big_group_indices:
			big_group_payloads.append(payload)
		else:
			non_big_group_paylaods.append(payload)
	
	# 시그니처 생성
	cluster_signatures = SG2(payloads=big_group_payloads, vector_size=vector_size, 
							 eps=eps, minpts=minpts, hh1_size=hh1_size, hh2_size=hh2_size, ratio=ratio)
	
	signatures = set()
	for value_list in cluster_signatures.values():
		for i in value_list:
			signatures.add(i[0])

	res = list(signatures)

	return res

In [11]:
with open('./datasets/pkl/pe_string.pkl', 'rb') as f:
    malware_strings = pickle.load(f)
    
with open('./datasets/pkl/pe_benign_string.pkl', 'rb') as f:
    benign_strings = pickle.load(f)

In [12]:
# split train test

train_ratio = 0.8

random.Random(42).shuffle(malware_strings)

train_data = malware_strings[: int(len(malware_strings) * train_ratio)]
test_data = malware_strings[int(len(malware_strings) * train_ratio): ]

print(len(malware_strings), len(train_data), len(test_data))

29909 23927 5982


In [13]:
# parameters

K = 64
M = 2 ** 14
thetaJ = 0.6
window_size = 4
vector_size = 512
eps = 0.4
minpts = 5
ngram = 4
hh1_size = 5000
hh2_size = 5000
hh3_size = 5000
ratio = 0.8

In [14]:
signatures = GIPS(str_feature=train_data, K=K, M=M, thetaJ=thetaJ, vector_size=vector_size,
     eps=eps, minpts=minpts, hh1_size=hh1_size, hh2_size=hh2_size, ratio=ratio
)

data no: 23927
make minhashed vector


100%|██████████| 23927/23927 [05:10<00:00, 77.10it/s] 


checking big group


100%|██████████| 23927/23927 [02:18<00:00, 172.69it/s]


chunking


100%|██████████| 18601/18601 [00:50<00:00, 369.23it/s]


start DBSCAN
end DBSCAN
make signature


100%|██████████| 4/4 [00:25<00:00,  6.43s/it]

end signature


In [15]:
sig_set = set(signatures)
sig_set

{'EStreamError',
 'lJt?Y9>',
 'Microsoft Corporation0',
 'user32.dll',
 'InternetCloseHandle',
 'comctl32.dll',
 '\\Magix Filme auf CD und DVD keygen.exe',
 '.rdata$zzzdbg',
 'xFf1Q"',
 'ekfJJH',
 'oOKrrE',
 'Microsoft I Service',
 '3I4M4Q4U4Y4]4a4e4i4m4q4u4y4}4',
 'UninstallString',
 'QQQQQQSVW3',
 'Primary',
 'Ymws;&',
 'RegOpenKeyA',
 'CloseServiceHandle',
 'eOKrrE',
 '\\System32\\progman.exe',
 'OJLNDwWK',
 'EL3N7`',
 '7*8J8Z8e8k8s8x8',
 'QTypInfo',
 '4M5Q5b5',
 '\\Alien Versus Predator EXE-NOCD.exe',
 '\\Magix Ringtone Maker [crack].exe',
 'CreateToolhelp32Snapshot',
 'uppvc"',
 'TlsSetValue',
 '343<3@3D3H3L3P3T3X3\\3`3d3h3l3x3',
 'KievvZc',
 'GetMessageA',
 "`eh vector copy constructor iterator'",
 'nan(ind)',
 'A KWZA',
 'wininet',
 '7087":gbkfJJH',
 '8s9S=A',
 'S,iC4gvZ',
 '$TwqwE',
 'lKJeJRfBynIfuyUawDMXPajKmCFKUTkvnANMKIybaeKGeiNoOFqts',
 '!.7z{(',
 'gbOWuuD',
 '\\Battlefield 1942 no cd crack.exe',
 '7$707:7A7K7R7\\7c7m7',
 '\\McAfee SpamKiller All Versions Crack.EXE',
 'o!W"

In [16]:
res = []

for strings in test_data:
    cnt = 0
    for string in strings:
        if string in sig_set:
            cnt += 1
    res.append(cnt)
    
print(res[: 100])
print(sum(res) / len(res))

[33, 4, 926, 33, 49, 13, 37, 248, 450, 20, 79, 3, 37, 107, 194, 60, 15, 19, 66, 926, 18, 92, 4, 41, 87, 92, 92, 19, 54, 41, 9, 143, 60, 41, 49, 92, 41, 245, 19, 245, 435, 49, 70, 25, 50, 435, 314, 20, 444, 906, 119, 49, 57, 92, 93, 208, 8, 15, 27, 33, 5, 223, 37, 41, 15, 49, 341, 46, 84, 33, 262, 90, 375, 84, 13, 13, 229, 18, 143, 15, 170, 38, 335, 15, 13, 49, 84, 84, 132, 283, 54, 84, 143, 52, 170, 37, 926, 143, 90, 22]
122.88900033433634


In [17]:
benign = []

for strings in benign_strings:
    cnt = 0
    for string in strings:
        if string in sig_set:
            cnt += 1
    benign.append(cnt)
    
print(benign[: 100])
print(sum(benign) / len(benign))

[218, 57, 69, 36, 61, 16, 2, 56, 268, 20, 133, 203, 102, 244, 344, 289, 304, 140, 34, 55, 111, 19, 3, 26, 46, 27, 7, 102, 127, 173, 234, 414, 264, 19, 248, 2, 2, 8, 244, 2, 107, 11, 16, 110, 18, 164, 7, 259, 72, 112, 109, 43, 30, 342, 59, 209, 89, 238, 322, 44, 17, 408, 42, 18, 9, 202, 11, 172, 224, 23, 21, 90, 8, 320, 1, 297, 4, 31, 22, 127, 23, 4, 93, 182, 194, 68, 2, 290, 2, 295, 5, 111, 8, 141, 92, 32, 37, 39, 46, 12]
111.56150322645358


In [18]:
with open('./res/PE_benign_signature.pkl', 'rb') as f:
    white_list_dict = pickle.load(f)
    
white_list_dict

{0: [('String', 37705),
  ('Thread', 30087),
  ('.idata', 25927),
  ('System', 23398),
  ('      ', 18535),
  ('A_A^A]', 18314),
  ('.rdata', 18130),
  ('A^A]A\\', 17569),
  ('Exception', 17560),
  ('Object', 16632),
  ('UAVAWH', 15923),
  ('Delete', 15212),
  ('AUAVAWH', 14549),
  ('A_A^A]A\\_', 14242),
  ('GetCurrentProcess', 13693),
  ('Version', 13181),
  ('handle', 12310),
  ('Environment', 11557),
  ('Console', 11461),
  (' delete', 11316),
  ('UnhandledExceptionFilter', 11055),
  ('WATAUAVAWH', 10351),
  ('0123456789', 10338),
  ('Strings', 9610),
  ('string', 9553),
  ('abcdefghijklmnopqrstuvwxyz', 9216),
  ('InitializeCriticalSection', 9035),
  ('ABCDEFGHIJKLMNOPQRSTUVWXYZ', 8595),
  ('FileName', 8594),
  ('ExitProcess', 8574),
  ('GetCurrentThread', 8133),
  ('GetWindow', 7878),
  ('A_A^A\\', 7852),
  ('connect', 7829),
  ('GetValue', 7692),
  ('length', 7363),
  ('!This program cannot be run in DOS mode.', 7281),
  ('.CRT$XIA', 7274),
  ('KERNEL32', 7201),
  ('Global', 7108)

In [19]:
white_list = set()

for key, value in white_list_dict.items():
    for string, cnt in value:
        white_list.add(string)
        
white_list

{'      ',
 '                          ',
 '      </requestedPrivileges>',
 '    </security>',
 '    <security>',
 '  </trustInfo>',
 ' !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`ABCDEFGHIJKLMNOPQRSTUVWXYZ{|}~',
 ' !"#$%&\'()*+,-./0123456789:;<=>?@abcdefghijklmnopqrstuvwxyz[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~',
 " Base Class Array'",
 ' Base Class Descriptor at (',
 " Class Hierarchy Descriptor'",
 " Complete Object Locator'",
 " Type Descriptor'",
 ' delete',
 ' delete[]',
 ' new[]',
 ' using a Numara Software, Inc. product.',
 '!This program cannot be run in DOS mode.',
 '(DigiCert SHA2 Assured ID Code Signing CA',
 '(null)',
 '+\\$$Ff',
 '.00cfg',
 '.?AVbad_alloc@std@@',
 '.?AVexception@std@@',
 '.?AVtype_info@@',
 '.CRT$XCA',
 '.CRT$XCZ',
 '.CRT$XIA',
 '.CRT$XIZ',
 '.CRT$XPA',
 '.CRT$XPX',
 '.CRT$XPZ',
 '.CRT$XTA',
 '.CRT$XTZ',
 '.SEFCMD',
 '.idata',
 '.idata$2',
 '.idata$3',
 '.idata$4',
 '.idata$5',
 '.idata$6',
 '.idata$7',
 '.pdata',
 '.rdata',
 '.rdata$

In [20]:
sig_set = set()

for i in signatures:
    if i not in white_list:
        sig_set.add(i)

In [21]:
for i in range(5, 13):
	res = []
	TP = 0
	FP = 0

	print('N', i)

	for strings in test_data:
		cnt = 0
		for string in strings:
			if string in sig_set:
				cnt += 1
		if cnt > i:
			TP += 1
		else:
			FP += 1
		res.append(cnt)
		
	# print(sum(res) / len(res))
	print(f'TP: {TP}, FP: {FP}')

	benign = []

	TN = 0
	FN = 0

	for strings in benign_strings[: 5000]:
		cnt = 0
		for string in strings:
			if string in sig_set:
				cnt += 1		
		if cnt <= i:
			TN += 1
		else:
			FN += 1
		benign.append(cnt)
		
	# print(sum(benign) / len(benign))
	print(f'TN: {TN}, FN: {FN}')

	presion = round(TP / (TP + FP) , 4)
	recall = round(TP / (TP + FN), 4)

	print(f'presion: {presion}, recall: {recall}')
	print('f1-score:', round(2 * (presion * recall) / (presion + recall), 4))
	print()

N 5
TP: 5681, FP: 301
TN: 1610, FN: 3390
presion: 0.9497, recall: 0.6263
f1-score: 0.7548

N 6
TP: 5664, FP: 318
TN: 1754, FN: 3246
presion: 0.9468, recall: 0.6357
f1-score: 0.7607

N 7
TP: 5355, FP: 627
TN: 1853, FN: 3147
presion: 0.8952, recall: 0.6299
f1-score: 0.7395

N 8
TP: 5245, FP: 737
TN: 1945, FN: 3055
presion: 0.8768, recall: 0.6319
f1-score: 0.7345

N 9
TP: 5120, FP: 862
TN: 2012, FN: 2988
presion: 0.8559, recall: 0.6315
f1-score: 0.7268

N 10
TP: 4905, FP: 1077
TN: 2073, FN: 2927
presion: 0.82, recall: 0.6263
f1-score: 0.7102

N 11
TP: 4767, FP: 1215
TN: 2136, FN: 2864
presion: 0.7969, recall: 0.6247
f1-score: 0.7004

N 12
TP: 4750, FP: 1232
TN: 2188, FN: 2812
presion: 0.794, recall: 0.6281
f1-score: 0.7014



# 문자열 추출 방법 개선

In [22]:
from GIPS.core.feature_extractor import extract_string_lower
import os
from tqdm import tqdm

In [23]:
malware_direc = './datasets/PE_malware/'
benign_direc = './datasets/ben/'

In [24]:
malware_path = os.listdir(malware_direc)

with open('./datasets/pkl/benign_md5.pkl', 'rb') as f:
    benign_path = pickle.load(f)

In [26]:
benign_strings = [] # 전체 스트링 저장
white_list = set() # train 화이트 리스트
benign_test = [] # test benign

for path_ in tqdm(benign_path[: int(len(benign_path) * 0.8)]):
	path = os.path.join(benign_direc, path_)
	
	res = extract_string_lower(path=path)
	benign_strings.append(res)
	white_list.update(res)

print(len(white_list))

with open('./datasets/pkl/white_list.pkl', 'wb') as f:
	pickle.dump(white_list, f)


for path_ in tqdm(benign_path[int(len(benign_path) * 0.8): ]):
	path = os.path.join(benign_direc, path_)
	
	res = extract_string_lower(path=path)
	benign_test.append(res)
	benign_strings.append(res)

with open('./datasets/pkl/benign_low_strings.pkl', 'wb') as f:
	pickle.dump(benign_strings, f)

100%|██████████| 23927/23927 [07:54<00:00, 50.41it/s]


24033730


100%|██████████| 5982/5982 [01:51<00:00, 53.48it/s]


In [27]:
malware_strings = [] # 전체스트링 저장
malware_train = [] # 실험 진행할 데이터
malware_test = [] # 테스트할 데이터
whitelist = set(white_list)

for path_ in tqdm(malware_path[: int(len(malware_path) * 0.8)]):
	path = os.path.join(malware_direc, path_)
	
	res = extract_string_lower(path=path)
	malware_strings.append(res)
	tmp = []
	for i in res:
		if i not in whitelist:
			tmp.append(i)
	malware_train.append(tmp)

print(len(white_list))


for path_ in tqdm(malware_path[int(len(malware_path) * 0.8): ]):
	path = os.path.join(malware_direc, path_)
	
	res = extract_string_lower(path=path)
	malware_strings.append(res)
	tmp = []
	for i in res:
		if i not in whitelist:
			tmp.append(i)
	malware_test.append(tmp)

with open('./datasets/pkl/malware_low_strings.pkl', 'wb') as f:
	pickle.dump(malware_strings, f)

100%|██████████| 23928/23928 [04:15<00:00, 93.61it/s] 


24033730


100%|██████████| 5982/5982 [01:04<00:00, 92.25it/s] 


In [28]:
# split train test

train_ratio = 0.8

random.Random(42).shuffle(malware_strings)

train_data = malware_strings[: int(len(malware_strings) * train_ratio)]
test_data = malware_strings[int(len(malware_strings) * train_ratio): ]

print(len(malware_strings), len(train_data), len(test_data))

29910 23928 5982


In [29]:
signatures = GIPS(str_feature=train_data, K=K, M=M, thetaJ=thetaJ, vector_size=vector_size,
     eps=eps, minpts=minpts, hh1_size=hh1_size, hh2_size=hh2_size, ratio=ratio
)

data no: 23928
make minhashed vector


100%|██████████| 23928/23928 [05:11<00:00, 76.78it/s] 


checking big group


100%|██████████| 23928/23928 [02:59<00:00, 133.02it/s]


chunking


100%|██████████| 18475/18475 [00:51<00:00, 361.15it/s]


start DBSCAN
end DBSCAN
make signature


100%|██████████| 3/3 [00:30<00:00, 10.13s/it]

end signature


In [30]:
sig_set = set(signatures)
sig_set

{'wnetenumresourcea',
 'sf:krg',
 'user32.dll',
 'sunday',
 'comctl32.dll',
 'getwindowthreadprocessid',
 'geticoninfo',
 'getprocesswindowstation',
 'r?}v06',
 '8l.q|df',
 '8n/s2#~',
 'xwwwwwwwwwwwwwwsssttpnjbllll',
 ',<"o1l',
 'mn6}pi',
 'module_check',
 '1%0g/"5f',
 'e[e7a6',
 'eheapexception',
 'clnone',
 'winmm.dll',
 '&f#$dl',
 '\\v8</{',
 '2!$el7',
 '=xp/yq',
 'uxtheme.dll',
 'wininet',
 '+90/p5t',
 'ar1ntinahpe',
 '=ec&jl/p',
 'kwindows',
 'tagcswi',
 'a>"u>m',
 'copyfilea',
 'eoverflow',
 'lyvn}tab',
 'getmodulehandlew',
 'b9huc$,9cg',
 '@isnec',
 '\\\\^yfz',
 "8'li?jl",
 '[q@m35',
 '?>bw:,',
 'clbtntext',
 'clmenutext',
 '&r8k~j',
 'clmoneygreen',
 'setfilepointer',
 '<?t#<*t',
 'estackoverflow',
 'realloc',
 'v`e&?b',
 '{^0kf<pq',
 'rintfa',
 'a/nb@;',
 '[dzsb>',
 '1#qnan',
 '!this is a pe executable',
 '\\l}eeuy+',
 'wsprintfa',
 'd$t#d$h',
 'globallock',
 'v0lc 33',
 'regopenkeyexa',
 'hangeul_charset',
 'bbbbbbbbbbbbbbb',
 'h"pdx`',
 'n_!a~)',
 'tele-brands.com',
 'loadli